In [1]:
!pip install -e ../../

Obtaining file:///C:/Users/netxph/Projects/mlpouch
  Attempting uninstall: mlpouch
    Found existing installation: mlpouch 0.0.1
    Uninstalling mlpouch-0.0.1:
      Successfully uninstalled mlpouch-0.0.1
  Running setup.py develop for mlpouch


In [2]:
import pandas as pd
from mlpouch.decomposition import FunkSVD
import logging

logging.basicConfig(level=logging.DEBUG)

In [3]:
movies = pd.read_csv("data/movies_clean.csv")
del movies['Unnamed: 0']
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
reviews = pd.read_csv("data/reviews_clean.csv")
del reviews['Unnamed: 0']
reviews.head()

In [ ]:
reviews_sub = reviews[["user_id", "movie_id", "rating"]]
reviews_sub.head()

In [ ]:
movie_reviews = reviews_sub.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
movie_reviews.head()

In [ ]:
movie_reviews_sub = movie_reviews[[73486, 75314,  68646, 99685]].dropna(axis=0)
movie_reviews_sub

In [ ]:
dec = FunkSVD(iters=10, learning_rate=0.005)
movie_reviews_pred = dec.fit_transform(movie_reviews_sub)

In [ ]:
movie_reviews_pred

In [4]:
from mlpouch.similarity import DotProduct
import numpy as np

In [5]:
movies_mat = movies.set_index("movie_id").iloc[:, 3:]
movies_mat.head()

,1800's,1900's,2000's,History,News,Horror,Musical,Film-Noir,Mystery,Adventure,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
movie_id,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
10,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
12,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
25,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
dp = DotProduct()
movies_dot = dp.fit_transform(movies_mat)
movies_dot.head()

DEBUG:root:Dot product done. Shape (31245, 31245).
DEBUG:root:Dataframe restructuring done.


,8,10,12,25,91,417,439,443,628,833,...,8144778,8144868,8206708,8289196,8324578,8335880,8342748,8342946,8402090,8439854
movie_id,,,,,,,,,,,,,,,,,,,,,
8,2,2,2,0,1,1,1,0,1,1,...,1,0,0,1,0,0,0,0,0,0
10,2,2,2,0,1,1,1,0,1,1,...,1,0,0,1,0,0,0,0,0,0
12,2,2,2,0,1,1,1,0,1,1,...,1,0,0,1,0,0,0,0,0,0
25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,1,1,1,0,2,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [32]:
similar_idxs = np.where(movies_dot[118661] == np.max(movies_dot[118661]))[0]
similar_movies = np.array(movies.iloc[similar_idxs,]["movie"])
print(len(similar_movies))
similar_movies

130


array(['Voyage to the Bottom of the Sea (1961)',
       'Beneath the Planet of the Apes (1970)',
       'Gojira tai Megaro (1973)', "Logan's Run (1976)",
       'The Spy Who Loved Me (1977)', 'Battlestar Galactica (1978)',
       'Buck Rogers in the 25th Century (1979)', 'Mad Max (1979)',
       'Moonraker (1979)', 'Flash Gordon (1980)', 'Mad Max 2 (1981)',
       'Firefox (1982)', 'TRON (1982)',
       'Spacehunter: Adventures in the Forbidden Zone (1983)',
       'Timerider: The Adventure of Lyle Swann (1982)', 'Dune (1984)',
       'The Last Starfighter (1984)',
       'Star Trek III: The Search for Spock (1984)',
       'Mad Max Beyond Thunderdome (1985)', 'Zone Troopers (1985)',
       'Steel Dawn (1987)', 'Future Hunters (1986)',
       'Star Trek V: The Final Frontier (1989)',
       'Star Trek VI: The Undiscovered Country (1991)',
       'The Fantastic Four (1994)', 'Stargate (1994)', 'GoldenEye (1995)',
       'Waterworld (1995)', 'Drive (1997)', 'Escape from L.A. (1996)',
   

In [28]:
movies[movies.movie.str.contains("Avengers")]

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
8700,118661,The Avengers (1998),Action|Adventure|Sci-Fi,1998,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
14913,848228,The Avengers (2012),Action|Sci-Fi|Thriller,2012,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
22956,2395427,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi,2015,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
27613,4154756,Avengers: Infinity War (2018),Action|Adventure|Fantasy,2018,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
27880,4296026,Avengers Grimm (2015),Action|Adventure|Fantasy,2015,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
29398,5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0
